In [7]:
model.n_features_in_

2

In [8]:
model.model_

PerFeatureTransformer(
  (encoder): SequentialEncoder(
    (0): RemoveEmptyFeaturesEncoderStep()
    (1): NanHandlingEncoderStep()
    (2): VariableNumFeaturesEncoderStep()
    (3): InputNormalizationEncoderStep()
    (4): VariableNumFeaturesEncoderStep()
    (5): LinearInputEncoderStep(
      (layer): Linear(in_features=4, out_features=192, bias=False)
    )
  )
  (y_encoder): SequentialEncoder(
    (0): NanHandlingEncoderStep()
    (1): LinearInputEncoderStep(
      (layer): Linear(in_features=2, out_features=192, bias=True)
    )
  )
  (transformer_encoder): LayerStack(
    (layers): ModuleList(
      (0-11): 12 x PerFeatureEncoderLayer(
        (self_attn_between_features): MultiHeadAttention()
        (self_attn_between_items): MultiHeadAttention()
        (mlp): MLP(
          (linear1): Linear(in_features=192, out_features=768, bias=False)
          (linear2): Linear(in_features=768, out_features=192, bias=False)
        )
        (layer_norms): ModuleList(
          (0-2): 3 x 

In [9]:
summary(model.model_)

Layer (type:depth-idx)                             Param #
PerFeatureTransformer                              --
├─SequentialEncoder: 1-1                           --
│    └─RemoveEmptyFeaturesEncoderStep: 2-1         --
│    └─NanHandlingEncoderStep: 2-2                 --
│    └─VariableNumFeaturesEncoderStep: 2-3         --
│    └─InputNormalizationEncoderStep: 2-4          --
│    └─VariableNumFeaturesEncoderStep: 2-5         --
│    └─LinearInputEncoderStep: 2-6                 --
│    │    └─Linear: 3-1                            768
├─SequentialEncoder: 1-2                           --
│    └─NanHandlingEncoderStep: 2-7                 --
│    └─LinearInputEncoderStep: 2-8                 --
│    │    └─Linear: 3-2                            576
├─LayerStack: 1-3                                  --
│    └─ModuleList: 2-9                             --
│    │    └─PerFeatureEncoderLayer: 3-3            589,824
│    │    └─PerFeatureEncoderLayer: 3-4            589,824
│    │    └

In [10]:
X_train.head(n).values

array([[2.83  , 5.19  ],
       [2.825 , 5.16  ],
       [2.82  , 5.13  ],
       [2.815 , 5.1   ],
       [2.81  , 5.07  ],
       [2.835 , 5.0525],
       [2.86  , 5.035 ],
       [2.885 , 5.0175],
       [2.91  , 5.    ],
       [2.9975, 5.025 ]])

In [11]:
import torch


cat_ix = []
style = None
X_full = X_train
X_full = torch.cat([torch.tensor(X_train.head(n).values, dtype=torch.float32), torch.tensor(X_validation.head(n).values, dtype=torch.float32)], dim=0).unsqueeze(1)

model.model_(
                    *(style, X_full, torch.tensor(y_train.head(n).values, dtype=torch.float32)),
                    only_return_standard_out=True,
                    categorical_inds=cat_ix,
                    single_eval_pos=n
                ).shape

torch.Size([10, 1, 5000])

In [12]:
torch.tensor(X_train.head(n).values, dtype=torch.float32)

tensor([[2.8300, 5.1900],
        [2.8250, 5.1600],
        [2.8200, 5.1300],
        [2.8150, 5.1000],
        [2.8100, 5.0700],
        [2.8350, 5.0525],
        [2.8600, 5.0350],
        [2.8850, 5.0175],
        [2.9100, 5.0000],
        [2.9975, 5.0250]])

In [25]:
for k in torch.tensor(X_train.head(n).values, dtype=torch.float32):
    print(torch.tensor(X_train.head(n).values, dtype=torch.float32)[k])

IndexError: tensors used as indices must be long, int, byte or bool tensors

Pipeline(steps=[('input_transformer', SafePowerTransformer()),
                ('standard',
                 Pipeline(steps=[('inf_to_nan_pre ',
                                  FunctionTransformer(check_inverse=False,
                                                      func=<function _inf_to_nan_func at 0x000001CA0EB959E0>,
                                                      inverse_func=<function _identity at 0x000001CA0EB95BC0>)),
                                 ('nan_impute_pre ',
                                  SimpleImputer(keep_empty_features=True)),
                                 ('standard', StandardScaler()),
                                 ('inf_to_nan_post',
                                  FunctionTransformer(check_inverse=False,
                                                      func=<function _inf_to_nan_func at 0x000001CA0EB959E0>,
                                                      inverse_func=<function _identity at 0x000001CA0EB95BC0>)),
                                 ('nan_impute_post',
                                  SimpleImputer(keep_empty_features=True))]))])

ColumnTransformer(sparse_threshold=0.0,
                  transformers=[('original', 'passthrough', [0, 1]),
                                ('feat_transform',
                                 QuantileTransformer(n_quantiles=2,
                                                     random_state=1894655253),
                                 [0, 1])])

n_components=max(1, min(num_examples // 10 + 1, num_features // 2))